# Perform a looping with the quadrotor

# Goal of the project

The goal of this project is to control a 2D quadrotor to get it to perform acrobatic moves. The controller will be designed using an SQP solver.

Please submit your code as a runnable Jupyter Notebook.

## 2D quadrotor

The quadrotor is depicted in the following figure
<img src='quadrotor.png' width="300">


The quadrotor model is written as
$$\begin{align} 
\dot{p_x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{p_y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $p_x$ is the horizontal and $p_y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $x = [p_x, v_x, p_y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the problem and provides all the useful information about the robot and methods to simulate and animate it as shown below.

You can access the different parameters of the model in the following way:

In [1]:
import quadrotor

print("Mass    =", quadrotor.MASS)
print("Length  =", quadrotor.LENGTH)
print("Inertia =", quadrotor.INERTIA)
print("Dt      =", quadrotor.DT)
print("state size   =", quadrotor.DIM_STATE)
print("control size =", quadrotor.DIM_CONTROL)

Mass    = 0.5
Length  = 0.15
Inertia = 0.1
Dt      = 0.04
state size   = 6
control size = 2




## Part 1 - Setting up the trajectory Optimization (50 points)
1. Discretize the system dynamics using the Euler method seen in class - write the time discretization step as $\delta t$ (use symbols not numbers for the mass, etc)
2. We would like the quadrotor to perfom a looping. Find and implement a suitable cost function to perform a looping and add constraint to maintain the thrust of each rotor between $0$ and $10$. Solve the problem using your own implementation of a SQP (levarging your code from Homework 2) with a large horizon to check that you can do a looping.
3. Show plots of all the states and controls of the robot as a function of time. Describe your design choices (in a concise manner) in the report.

## Part 2 - Model predictive control (MPC) (50 points)
1. Use the trajectory optimization method from Part I to design a MPC controller and test it using the simulator below. In particular, verify that it can handle perturbations by calling the ```quadrotor.simulate``` function with ```disturbance = True``` (when setting disturbance to True, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state anc control evolution.
2. Explain your intended design in the report, including the cost function and found control law

The ```quadrotor.simulate``` function takes as an input an initial state, a controller, the number of discrete time steps and a boolean value to indicate the presence of perturbation. The controller has to be a function taking as an input a state and time index and outputting a control vector.

To visualize the trajectory, use the ```quadrotor.animate_robot``` function
and show the animation (show the plots in your report).

## Bonus (10 points)
Add a state constraint to perform the looping while maintening a positive altitude. Use the origin as an initial state.

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython
from scipy.linalg import block_diag
from scipy.sparse import csr_matrix
from qpsolvers import solve_qp, Problem, solve_problem

def controller(x, t):
    return np.zeros(2)

x_init = np.array([0, 0, 0., 0 ,0, 0])
horizon_length = 100
t, state, u = quadrotor.simulate(x_init, controller, horizon_length, disturbance=True)
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.

## Part 1
1. Given State dynamics:
$$\begin{align} 
\dot{p_x} &= v_x\\
\dot{v}_x &= - \frac{(u_1 + u_2) \sin \theta}{m} \\ 
\dot{p_y} &= v_y\\
\dot{v}_y &= \frac{(u_1 + u_2) \cos \theta}{m}  - g\\
\dot{\theta} &= \omega\\
\dot{\omega} &= \frac{r (u_1 - u_2)}{I} \end{align}$$

After Discretizing the state dynamics and doing Taylor Series Expansion to get a linear approximation
$$\begin{align}
    \delta p_{x_{n+1}} &= \delta p_{x_n} + \delta t \delta v_{x_n} \\
    \delta v_{x_{n+1}} &= \delta v_{x_n} - \delta t \delta \theta_n \frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} - \delta t (\delta u_{1_n} +  \delta u_{2_n} )\frac{\sin \theta_n}{m}\\
    \delta p_{y_{n+1}} &= \delta p_{y_n} + \delta t \delta v_{y_n} \\
    \delta v_{y_{n+1}} &= \delta v_{y_n} - \delta t \delta \theta_n \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} + \delta t (\delta u_{1_n} + \delta u_{2_n}) \frac{\cos \theta_n}{m}\\
    \delta \theta_{n+1} &= \delta \theta_{n} + \delta t \delta \omega_n \\
    \delta \omega_{n+1} &= \delta \omega_{n} + \delta t \frac{L(\delta u_{1_n} - \delta u_{2_n})}{I}
\end{align}$$

In [ ]:
class Optimization:
    # initialize the variables from the module quadrotor
    def __init__(self, m, l, I, dt, xdim, udim):
        # Initialize the variables from the quadrotor module
        self.m = m
        self.l = l
        self.I = I
        self.dt = dt
        self.xdim = xdim
        self.udim = udim
        

    # define the quadrotor dynamics
    def quadrotor_dynamics(self, x_init, y, N=100):
        nvar = self.xdim + self.udim
        # Typecast to NumPy arrays as a precaution
        y = np.asarray(y, dtype=np.float64)
        x_init = np.asarray(x_init, dtype=np.float64)

        # Flatten the arrays just in case
        y = y.flatten()
        x_init = x_init.flatten()

        # Print the shapes
        # print(f"Total shape of y: {y.shape}")
        # print(f"Total shape of x_init: {x_init.shape}")

        # Define the placeholders for A and b
        A = np.zeros((6 * N, nvar * N))
        b = np.zeros(6 * N)

        # Define the initial state constraints
        A[:6, :] = np.eye(6, nvar * N)
        b[:6] = x_init - y[:6]

        # Define the constraints for the rest of the time steps
        '''
        Check again the range of the loop N or N-1 
        '''
        for t in range(N-1):
            # Propogate the indices
            i = t * nvar
            n = i
            n1 = i + nvar

            # Assign the variables at time step t
            px = y[n]
            vx = y[n + 1]
            py = y[n + 2]
            vy = y[n + 3]
            theta = y[n + 4]
            omega = y[n + 5]
            u1 = y[n + 6]
            u2 = y[n + 7]

            # Assign the variables at time step t+1
            px_1 = y[n1]
            vx_1 = y[n1 + 1]
            py_1 = y[n1 + 2]
            vy_1 = y[n1 + 3]
            theta_1 = y[n1 + 4]
            omega_1 = y[n1 + 5]

            # Position x constraint
            A[6 * (t+1), n] = 1
            A[6 * (t+1), n + 1] = self.dt
            A[6 * (t+1), n1] = -1
            b[6 * (t+1)] = px + self.dt * vx - px_1

            # Velocity x constraint
            A[6 * (t+1) + 1, n + 1] = 1
            A[6 * (t+1) + 1, n + 4] = -self.dt * ((u1 + u2) * np.cos(theta) / self.m)
            A[6 * (t+1) + 1, n + 6] = -self.dt * (np.sin(theta) / self.m)
            A[6 * (t+1) + 1, n + 7] = -self.dt * (np.sin(theta) / self.m)
            A[6 * (t+1) + 1, n1 + 1] = -1
            b[6 * (t+1) + 1] = vx - self.dt * (u1 + u2) * np.sin(theta) / self.m - vx_1

            # Position y constraint
            A[6 * (t+1) + 2, n + 2] = 1
            A[6 * (t+1) + 2, n + 3] = self.dt
            A[6 * (t+1) + 2, n1 + 2] = -1
            b[6 * (t+1) + 2] = py + self.dt * vy - py_1
            
            # Velocity y constraint
            A[6 * (t+1) + 3, n + 3] = 1
            A[6 * (t+1) + 3, n + 4] = -self.dt * ((u1 + u2) * np.sin(theta) / self.m)
            A[6 * (t+1) + 3, n + 6] = self.dt * (np.cos(theta) / self.m)
            A[6 * (t+1) + 3, n + 7] = self.dt * (np.cos(theta) / self.m)
            A[6 * (t+1) + 3, n1 + 3] = -1
            b[6 * (t+1) + 3] = vy + self.dt * (((u1 + u2) * np.cos(theta) / self.m) - 9.81) - vy_1

            # Angle theta constraint
            A[6 * (t+1) + 4, n + 4] = 1
            A[6 * (t+1) + 4, n + 5] = self.dt
            A[6 * (t+1) + 4, n1 + 4] = -1
            b[6 * (t+1) + 4] = theta + self.dt * omega - theta_1

            # Angular velocity omega constraint
            A[6 * (t+1) + 5, n + 5] = 1
            A[6 * (t+1) + 5, n + 6] = self.dt * self.l / self.I
            A[6 * (t+1) + 5, n + 7] = -self.dt * self.l / self.I
            A[6 * (t+1) + 5, n1 + 5] = -1
            b[6 * (t+1) + 5] = omega + self.dt * self.l * (u1 - u2) / self.I - omega_1

        # # Print shapes of A and b
        # print(f"Shape of A: {A.shape}")
        # print(f"Shape of b: {b.shape}")
        return A, b
    
    def circle_traj_gen(self, x_init, k, N, radius=3, y_max=3, x_max=1):
        # Initialize the variables
        x_init = np.asarray(x_init).flatten()

        # Define the circle trajectory
        if k<=N/5:
            des_x = 0
            des_y = 0
            des_theta = 0
        elif (N/5)<k<=2*(N/5):
            des_x = x_max
            des_y = x_max
            des_theta = np.pi/2
        elif 2*(N/5)<k<=3*(N/5):
            des_x = 0
            des_y = y_max
            des_theta = np.pi
        elif 3*(N/5)<k<=4*(N/5):
            des_x = -x_max
            des_y = y_max
            des_theta = 3*np.pi/2
        elif 4*(N/5)<k<=N:
            des_x = 0
            des_y = 0
            des_theta = 2*np.pi

        
        return des_x, des_y, des_theta
    
    def lissajous_traj_gen(self, k, N, y_max=3, x_max=1):
        # Initialize the variables
        x_init = np.asarray(x_init).flatten()

        # Define the liassajous trajectory
        if k<=N/10:
            des_x = 0
            des_y = 0
            des_theta = 0
        elif (N/10)<k<=2*(N/10):
            des_x = x_max
            des_y = x_max
            des_theta = np.pi/2
        elif 2*(N/10)<k<=3*(N/10):
            des_x = 0
            des_y = y_max
            des_theta = np.pi
        elif 3*(N/10)<k<=4*(N/10):
            des_x = -x_max
            des_y = y_max
            des_theta = 3*np.pi/2
        elif 4*(N/10)<k<=5*(N/10):
            des_x = 0
            des_y = 0
            des_theta = 2*np.pi
        elif 5*(N/10)<k<=6*(N/10):
            des_x = x_max
            des_y = -x_max
            des_theta = 3*np.pi/2
        elif 6*(N/10)<k<=7*(N/10):
            des_x = 0
            des_y = -y_max
            des_theta = np.pi
        elif 7*(N/10)<k<=8*(N/10):
            des_x = -x_max
            des_y = -x_max
            des_theta = np.pi/2
        elif 8*(N/10)<k<=9*(N/10):
            des_x = 0
            des_y = 0
            des_theta = 0
        elif 9*(N/10)<k<=N:
            des_x = 0
            des_y = 0
            des_theta = 0

        return des_x, des_y, des_theta

    
    def cost_function(self, y, x_init, N):
        # Number of variables in state and control
        nvar = self.xdim + self.udim
        # Typecast to NumPy arrays and flatten them as a precaution
        y = np.asarray(y, dtype=np.float64).flatten()
        x_init= np.asarray(x_init, dtype=np.float64).flatten()
        mini_Q = np.diag([10, 1, 10, 1, 10, 1])
        mini_R = np.diag([0.1, 0.1])

        # Initialize the cost for N states and N-1 controls
        P = block_diag(*([mini_Q, mini_R] * N))
        q = np.zeros((nvar * N)).flatten()

        

        # Loop over the horizon
        for iteration in range(N):
            des_x, des_y, des_theta = self.circle_traj_gen(x_init, k=iteration, N=N)
            q[(nvar * iteration):(nvar * iteration + nvar)] = np.concatenate(((-(np.array([[des_x], [0], [des_y], [0], [des_theta], [0]]).T @ mini_Q)).flatten(), [0, 0]))

        q = q.reshape(-1, 1).flatten()
        cost = 0.5 * y.T @ P @ y + 2*q.T @ y

        # Sanity Check for the shapes
        print(f"Shape of P: {P.shape}")
        print(f"Shape of q: {q.shape}")
        print(f"Shape of y: {y.shape}")
        print(f"Shape of x_init: {x_init.shape}")
        print(f"Cost: {cost[0].shape}")


        return cost[0]

    def grad_cost(self, x_init, y, N):
        # Number of variables in state and control
        nvar = self.xdim + self.udim

        # Typecast to NumPy arrays and flatten them as a precaution
        x_init = np.asarray(x_init, dtype=np.float64).flatten()
        y = np.asarray(y, dtype=np.float64).flatten()
        mini_Q = np.diag([10, 1, 10, 1, 10, 1])
        mini_R = np.diag([0.1, 0.1])

        P = block_diag(*([mini_Q, mini_R] * N))
        q = np.zeros((nvar * N)).flatten()

        for iteration in range(N):
            des_x, des_y, des_theta = self.circle_traj_gen(x_init, k=iteration, N=N)
            q[(nvar * iteration):(nvar * iteration + nvar)] = np.concatenate(((-(np.array([[des_x], [0], [des_y], [0], [des_theta], [0]]).T @ mini_Q)).flatten(), [0, 0]))

        calc_grad = P @ y + 2*q

        return calc_grad
    
    def hess_cost(self, x_init, y, N):
        mini_Q = np.diag([10, 1, 10, 1, 10, 1])
        mini_R = np.diag([0.1, 0.1])

        cost_hess = block_diag(*([mini_Q, mini_R] * N))

        return cost_hess
    
    def equality_cons(self, x_init, y, N):
        # Number of variavles in the state and control
        nvar = self.xdim + self.udim
        # Typecast to NumPy arrays and flatten them as a precaution
        y = np.asarray(y, dtype=np.float64).flatten()

        mini_G = np.array([[0, 0, 0, 0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 0, 0, 0, 1],
                           [0, 0, 0, 0, 0, 0, -1, 0],
                           [0, 0, 0, 0, 0, 0, 0, -1]])
        
        G = block_diag(*([mini_G] * N))
        h = np.zeros((4 * N)).flatten()

        for i in range(N):
            u1 = y[(nvar * i) + 6]
            u2 = y[(nvar * i) + 7]

            h[((i * self.udim * 2))] = 10 - u1
            h[((i * self.udim * 2) + 1)] = -u1
            h[((i * self.udim * 2) + 2)] = 10 - u2
            h[((i * self.udim * 2) + 3)] = u2

        return G, h
    

    def KKT_Solver(self, x_init, y, N):
        # Compute the 
        A, b = self.quadrotor_dynamics(x_init, y, N)
        A = csr_matrix(A)
        G, h = self.equality_cons(x_init, y, N)
        G = csr_matrix(G)
        kkt_prob = Problem(self.hess_cost(x_init, y, N), self.grad_cost(x_init, y, N), A, b, G, h)

        try:
            kkt_sol = solve_problem(problem=kkt_prob, solver="cvxopt")
            if kkt_sol is None:
                print("No solution found")
        except Exception as e:
            print(f"Exception: {e}")
            print("Solver Failed")

        return kkt_sol
    
    def constaint_violation(self, x_init, y, N):
        nvar = self.xdim + self.udim
        y = np.asarray(y, dtype=np.float64).flatten()
        x_init = np.asarray(x_init, dtype=np.float64).flatten()

        _, b = self.quadrotor_dynamics(x_init, y, N)
        total_violation = np.sum(np.abs(b))
        inequality_violation = 0

        for i in range(N):
            u1 = y[(nvar * i) + 6]
            u2 = y[(nvar * i) + 7]
            if u1 > 10:
                total_violation += np.abs(u1 - 10)
            elif u1 < 0:
                total_violation += np.abs(u1)

            if u2 > 10:
                total_violation += np.abs(u2 - 10)
            elif u2 < 0:
                total_violation += np.abs(u2)

        return total_violation + inequality_violation
    
    def line_search(self, x_init, est_y, N, alpha=0.9, rho=0.5, tol=1e-4, iterations=1000):
        # Typecast to NumPy arrays as a precaution
        est_y = np.asarray(est_y, dtype=np.float64).flatten()
        x_init = np.asarray(x_init, dtype=np.float64).flatten()

        # Initialize the variables
        nvar = self.xdim + self.udim
        min_violation = np.inf
        optimal_cost = np.inf
        cost_list = []
        alpha_list = []
        violation_list = []

        for i in range(iterations):
            sol = self.KKT_Solver(x_init, est_y, N)
            pk = sol.x
            curr_cost = self.cost_function(y=(est_y+(alpha*pk)), x_init=x_init, N=N)
            violation = self.constaint_violation(x_init=x_init, y=(est_y+(alpha*pk)), N=N)
            
            while (curr_cost >= optimal_cost) and violation >= min_violation:
                alpha = alpha * rho
                if alpha < 1e-8:
                    print("Small alpha")
                    break
            
            min_violation = violation
            optimal_cost = curr_cost
            cost_list.append(optimal_cost)
            alpha_list.append(alpha)
            violation_list.append(min_violation)

            print(f"Iteration: {i}, Cost: {optimal_cost}, Violation: {min_violation}, Alpha: {alpha}")

            est_y = est_y + alpha * pk
            if min_violation < tol:
                print("Convergence reached")
                break
            
        est_y = est_y.reshape(-1, 8)
        x = est_y[:, :6].T
        u = est_y[:, 6:].T
        return x, u



In [8]:
# Initialize the optimization class
opt = Optimization(quadrotor.MASS, quadrotor.LENGTH, quadrotor.INERTIA, quadrotor.DT, quadrotor.DIM_STATE, quadrotor.DIM_CONTROL)
N = 100
nvar = quadrotor.DIM_STATE + quadrotor.DIM_CONTROL
# Initialize the variables
x_init = np.array([0, 0, 0, 0, 0, 0])
y = np.zeros(nvar * N)
# Call the line search function
x, u = opt.line_search(x_init, y, N, alpha=0.9, rho=0.5, tol=1e-4, iterations=1000)


Total shape of y: (800,)
Total shape of x_init: (6,)
Shape of A: (600, 800)
Shape of b: (600,)
Exception: Rank(A) < p or Rank([P; A; G]) < n
Solver Failed


UnboundLocalError: local variable 'kkt_sol' referenced before assignment